In [1]:
# !pip install keras --upgrade
# uncomment the above line if you don't have keras 3 installed
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
# Preprocessing step
# TODO
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop(columns=['Unnamed: 0'], inplace=True)
train['satisfaction'].replace({'neutral or dissatisfied': 0, 'satisfied': 1}, inplace=True)
test.drop(columns=['Unnamed: 0'], inplace=True)

train.head()

In [3]:
print("\nMissing values in training data (before handling):")
print(train.isnull().sum())

print("\nMissing values in test data (before handling):")
print(test.isnull().sum())

In [4]:
mean = train.groupby('Departure/Arrival time convenient')['Arrival Delay in Minutes'].transform('mean')

train['Arrival Delay in Minutes'].fillna(mean.astype('int64'), inplace = True)
test['Arrival Delay in Minutes'].fillna(mean.astype('int64'), inplace = True)

In [5]:
print("\nMissing values in training data (after handling):")
print(train.isnull().sum())

print("\nMissing values in test data (after handling):")
print(test.isnull().sum())

In [6]:
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

train = pd.get_dummies(train, columns=categorical_cols, drop_first=True).replace({False: 0, True: 1})
test = pd.get_dummies(test, columns=categorical_cols, drop_first=True).replace({False: 0, True: 1})

In [7]:
train.head()

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns

# List of rating columns
rating_cols = [
    'Inflight wifi service',
    'Departure/Arrival time convenient',
    'Ease of Online booking',
    'Gate location',
    'Food and drink',
    'Online boarding',
    'Seat comfort',
    'Inflight entertainment',
    'On-board service',
    'Leg room service',
    'Baggage handling',
    'Checkin service',
    'Inflight service',
    'Cleanliness'
]
average_rating = pd.DataFrame(train[rating_cols].mean(axis=1), columns=['Average Rating'])

numerical_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes'] 

plt.figure(figsize=(10, 8))
sns.heatmap(pd.concat([train[numerical_cols], average_rating, train['satisfaction']], axis=1).corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train_cleaned.drop(columns=['satisfaction']), train_cleaned['satisfaction'], test_size=0.2, random_state=42)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train.drop(columns=['satisfaction']), train['satisfaction'], test_size=0.2, random_state=42)

In [11]:
X_train.shape

In [12]:
# Model design
# TODO

model = keras.Sequential()
model.add(keras.layers.Input(shape=(X_train.shape[1], )))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(128, activation='selu', kernel_initializer=keras.initializers.LecunNormal()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(64, activation='selu', kernel_initializer=keras.initializers.LecunNormal()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(32, activation='selu', kernel_initializer=keras.initializers.LecunNormal()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.LecunNormal()))

In [13]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [14]:
model.summary()

In [15]:
epochs = 100
BATCH_SIZE = 64

early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epochs, batch_size=BATCH_SIZE, callbacks=[early_stopping_cb])

In [16]:
# Evaluate
# TODO

from sklearn.metrics import accuracy_score

# TODO: Evaluate Model
val_predictions = model.predict(X_valid, batch_size=64)
print(accuracy_score(y_valid, val_predictions))

In [17]:
# Evaluate
# TODO

from sklearn.metrics import accuracy_score

# TODO: Evaluate Model
val_predictions = model.predict(X_valid, batch_size=64)
print(val_predictions)
print(accuracy_score(y_valid, val_predictions))

In [18]:
# Evaluate
# TODO

from sklearn.metrics import accuracy_score

# TODO: Evaluate Model
val_predictions = np.where(model.predict(X_valid, batch_size=64) > 0.5, 1, 0)
print(accuracy_score(y_valid, val_predictions))

In [19]:
# Preparing submission file
submission = val_predictions = np.where(model.predict(test, batch_size=64) > 0.5, 1, 0) # TODO
submission

In [20]:
# Preparing submission file
submission = pd.DataFrame(val_predictions = np.where(model.predict(test, batch_size=64) > 0.5, 1, 0), columns=['satisfaction']) # TODO
submission

In [21]:
# Preparing submission file
submission = pd.DataFrame(np.where(model.predict(test, batch_size=64) > 0.5, 1, 0), columns=['satisfaction']) # TODO
submission

In [22]:
# Preparing submission file
submission = pd.DataFrame(
    np.where(model.predict(test, batch_size=64) > 0.5, 1, 0), columns=['satisfaction']
    ).replace({0 : 'neutral or dissatisfied', 1 : 'satisfied'}) # TODO
submission